## Домашнее задание к семинару 7.
Составьте план эксперимента для 2-х любых А/В-тестов (можно выбрать продукты из первых вебинаров). Важно избежать Peeking Problem, гипотезы прописывать по структуре. 
Распишите, какие метрики и как вы будете анализировать. Используйте шаблон эксперимента из лекции и ссылку "Помощь в ДЗ".

In [4]:
from statsmodels.stats.power import tt_ind_solve_power, zt_ind_solve_power
from statsmodels.stats.proportion import proportion_effectsize
from statsmodels.stats.meta_analysis import effectsize_smd
from typing import Union
import plotly.graph_objects as go
from scipy import stats
from math import asin
import numpy as np

### Дизайн эксперимента : 
1. **Гипотеза**:	Перемещение на верх сайта ozon.ru блока с информацией о скидках повысит конверсию на 30%, потому что пользователь сможет быстрее найти все сезонные товары на сайте.   
   
   
2. **Что делаем**:
    - Контрольная группа: Оставляем всё без изменений.
    - Тестовая группа: Перемещаем блок информаци о скидках на верх первого экрана.    
       
       
3. **На каких пользователях тестируем**:	На всех пользователях сайта на **15%** трафика. Рандомно делим пользователей на две группы, контрольную и тестовую, для оценки влияния программы на увеличение оборота.   
 
 
4. **Ключевые метрики для оценки эксперимента**:
    - CR - primary; 
    - Revenue - secondary. 
    
**Длительность эксперимента 2 дня** (расчёт ниже)
       
       
5. **Ожидаемый эффект**:	увеличение количества покупок (конверсий) на 30%.  
 
 
6. **План действий в зависимости от результатов эксперимента**:	
    - При положительном результате эксперимента (в тестовой группе конверсии увеличиваются на 30%) расширяем действие программы на всех пользователей сайта;
    - Если результат не окажет статистически значимого эффекта на увеличение конверсий, останавливаем эксперимент.

Рассчитаем время проведения эксперимента:

Исторические данные: 
1. Средний трафик 5000 человек в день. 
2. Конверсия 4%.
4. Ожидаемый эффект 30%
5. alpha = 5% и beta = 20%

In [16]:
# исходные данные
DAU = 5000 
conv_1 = 0.04
delta = 0.3
conv_2 = conv_1 + conv_1 * delta
conv_2

0.052000000000000005

По имеющимся данным рассчитаем размер выборки для оценки изменений:
1. Найдем effect_size для дискретной метрики:

In [11]:
def calc_proportion_es(prob1: float, prob2: float):
    
    return abs(proportion_effectsize(prob1, prob2))

In [12]:
es_prop = calc_proportion_es(conv_1, conv_2)
es_prop

0.05740239580200046

2. Рассчитаем минимально необходимую выборку для полученного effectsize 

In [13]:
def calc_sample_size_proportion(effect_size: float, 
                              alpha: float = 0.05,
                              beta: float = 0.2,
                              ratio: Union[float, int] = 1):
    
    n = zt_ind_solve_power(effect_size=effect_size, 
                              alpha=alpha,
                              power=1-beta,
                              ratio=ratio,
                          )
    return int(n * 2)

In [14]:
print(f'Минимальное количество пользователей сайта, необходимое для проверки результатов эксперимента:{calc_sample_size_proportion(es_prop)}')

Минимальное количество пользователей сайта, необходимое для проверки результатов эксперимента:9528


3. Рассчитаем длительность эксперимента в днях.    

Для этого полученное значение минимально необходимой выборки разделим на количество пользователей, посещающих сайт в день:

In [15]:
print(f'Количество дней эксперимента: {calc_sample_size_proportion(es_prop) / DAU}')

Количество дней эксперимента: 1.9056
